In [1]:
from pyopenms import *
import pandas as pd
import numpy as np
import os
import glob

input_mzml_files = sorted(glob.glob('results/interim/PCpeak_*.mzML'))

raw_data=[]
for filename in input_mzml_files:
    exp= MSExperiment()
    MzMLFile().load(filename, exp)
    exp.sortSpectra(True)
    raw_data.append(exp)

ref_index = [i[0] for i in sorted(enumerate([exp.size() for exp in raw_data]), key=lambda x:x[1])][-1]
aligner = MapAlignmentAlgorithmPoseClustering()
aligner_par= aligner.getDefaults()
aligner_par.setValue("max_num_peaks_considered", 3000)
aligner.setParameters(aligner_par)
aligner.setReference(raw_data[ref_index])

for exp in raw_data[:ref_index] + raw_data[ref_index+1:]:
    trafo = TransformationDescription()
    aligner.align(exp, trafo)
    transformer = MapAlignmentTransformer()
    transformer.transformRetentionTimes(exp, trafo, True)

for exp in raw_data:    
    mzml_file = os.path.join("results", "", "Requant", "", "interim", "", 'MapAligned_' + os.path.basename(exp.getLoadedFilePath())[7:-5] +".mzML")
    MzMLFile().store(mzml_file, exp)

Determination of memory status is not supported on this 
 platform, measuring for memoryleaks will never fail


In [2]:
import csv
# read tsv file and create list of FeatureFinderMetaboIdentCompound
def metaboTableFromFile(path_to_library_file):
    metaboTable = []
    with open(path_to_library_file, 'r') as tsv_file:
        tsv_reader = csv.reader(tsv_file, delimiter="\t")
        next(tsv_reader) # skip header
        for row in tsv_reader:
            metaboTable.append(FeatureFinderMetaboIdentCompound(
                row[0], # name
                row[1], # sum formula
                float(row[2]), # mass
                [int(charge) for charge in row[3].split(',')], # charges
                [float(rt) for rt in row[4].split(',')], # RTs
                [float(rt_range) for rt_range in row[5].split(',')], # RT ranges
                [float(iso_distrib) for iso_distrib in row[6].split(',')] # isotope distributions
            ))
    return metaboTable

In [2]:
from pyopenms import *
import pandas as pd
import numpy as np
import os
import glob

Determination of memory status is not supported on this 
 platform, measuring for memoryleaks will never fail


In [8]:
input_mzml_files=sorted(glob.glob("results/Requant/interim/*.mzML"))
# load ms data from mzML file into MSExperiment
for mzml_file in input_mzml_files:
    spectra = MSExperiment()
    MzMLFile().load(mzml_file, spectra)

    # create FeatureFinderAlgorithmMetaboIdent and assign ms data
    ff = FeatureFinderAlgorithmMetaboIdent()
    ff.setMSData(spectra)

    # read library generate a metabo table with compounds
    metabo_table = metaboTableFromFile('resources/MetaboliteIdentification.tsv')

    params = ff.getParameters()
    params[b'extract:mz_window'] = 5.0 
    params[b'detect:peak_width'] = 20.0  #adjust for wide peaks
    ff.setParameters(params)
    # FeatureMap to store results
    fm = FeatureMap()
    # run the FeatureFinderMetaboIdent with the metabo_table and store results in fm
    ff.run(metabo_table, fm, String(mzml_file))

    # save FeatureMap to file
    ff_file = os.path.join("results", "", "Requant", "", "interim", "", 'FFMI_' + os.path.basename(mzml_file)[11:-5] +".featureXML")
    FeatureXMLFile().store(ff_file, fm)

RT window size calculated as 80 seconds.
<RT window size calculated as 240 seconds.> occurred 2 times
Extracting chromatograms...
Detecting chromatographic peaks...
Found 12189 feature candidates in total.
5591 features left after selection of best candidates.
4990 features left after resolving overlaps (involving 1134 features in 529 groups).
Model fitting: 2450 successes, 2540 failures
Imputing model failures with a linear model based on log(rawIntensities). Slope: 0.895544, Intercept: 1.75283
RT window size calculated as 240 seconds.
RT window size calculated as 80 seconds.
Extracting chromatograms...
Detecting chromatographic peaks...
Found 10824 feature candidates in total.
5097 features left after selection of best candidates.
4467 features left after resolving overlaps (involving 1182 features in 544 groups).
Model fitting: 1842 successes, 2625 failures
Imputing model failures with a linear model based on log(rawIntensities). Slope: 0.872118, Intercept: 1.93743
RT window size ca

In [9]:
from collections import defaultdict
from functools import reduce
from pathlib import Path
from time import perf_counter
import sys

from IPython.core.display import display
from pandas import CategoricalDtype
import numpy as np
from pyopenms import *
import pandas as pd
import os

common_meta_value_types = {
    b'label': 'U30',
    b'spectrum_index': 'i',
    b'score_fit': 'f',
    b'score_correlation': 'f',
    b'FWHM': 'f',
    b'spectrum_native_id': 'U30',
    b"num_of_masstraces" : "f"
}

class FeatureMapDF(FeatureMap):
    def __init__(self):
        super().__init__()
    
    # meta_values = None (default), 'all' or list of meta value names
    def get_df(self, meta_values = None):
        # get all possible meta value keys in a set
        if meta_values == 'all':
            meta_values = set()
            for f in self:
                mvs = []
                f.getKeys(mvs)
                for m in mvs:
                    meta_values.add(m)
        elif not meta_values: # if None, set to empty list
            meta_values = []
        
        def gen(fmap: FeatureMap, fun):
            for f in fmap:
                yield from fun(f, meta_values)

        def extract_meta_data(f: Feature, meta_values):
            bb = f.getConvexHull().getBoundingBox2D()
                
            vals = [f.getMetaValue(m) if f.metaValueExists(m) else np.NA for m in meta_values]   # find some NA or None value for numpy

            yield f.getUniqueId(), f.getCharge(), f.getRT(), f.getMZ(), bb[0][0], bb[1][0], f.getOverallQuality(), f.getIntensity()

        cnt = self.size()

        mddtypes = [('id', np.dtype('uint64')), ('charge', 'i4'), ('RT', 'f'), ('mz', 'f'),
                    ('RTstart', 'f'), ('RTend', 'f'),
                    ('quality', 'f'), ('intensity', 'f')]
        
        for meta_value in meta_values:
            if meta_value in common_meta_value_types:
                mddtypes.append((meta_value.decode(), common_meta_value_types[meta_value]))
            else:
                mddtypes.append((meta_value.decode(), 'U50'))
        mdarr = np.fromiter(iter=gen(self, extract_meta_data), dtype=mddtypes, count=cnt)
        df= pd.DataFrame(mdarr).set_index('id').sort_values("mz")
        #df= df[df["num_of_masstraces"]>=2]
        return df

In [10]:
input_feature_files = sorted(glob.glob('results/Requant/interim/FFMI_*.featureXML'))

for filename in input_feature_files:
    fmap = FeatureMapDF()
    FeatureXMLFile().load(filename, fmap)
    DF= fmap.get_df()
    feature_csv= os.path.join("results", "", "Requant", "", 'features_' +  os.path.basename(fmap.getMetaValue('spectra_data')[0].decode())[11:-5] +".csv")
    DF.to_csv(feature_csv)
    print(os.path.basename(filename))
    display(DF)

FFMI_20210827_UMETAB219_POS_DNPM_Plate-2_MDNAWGS14_blank.featureXML


,charge,RT,mz,RTstart,RTend,quality,intensity
id,,,,,,,
6779460035304800368,1,64.865883,152.056168,57.401688,71.244118,2.771286,2.030070e+05
13943752185986203656,1,220.469421,152.070084,213.578094,228.373856,3.778659,3.782497e+04
12798573101193454331,1,64.865891,155.080963,57.401688,71.244118,4.814148,2.494702e+05
466348627699078712,1,105.925674,155.080978,98.692268,113.328033,4.803148,2.377574e+05
5681079727169575490,1,64.856339,157.096588,59.941105,71.244118,2.600025,4.599726e+04
...,...,...,...,...,...,...,...
5498814147990939282,3,57.570805,1482.843018,51.029140,58.682755,3.974073,1.321363e+05
4782236559612104679,3,65.550865,1483.177979,61.170502,73.740509,3.552404,2.490552e+05
8137237597188825654,2,56.711224,1485.514160,48.486141,62.378098,4.067296,6.274994e+05


FFMI_20210827_UMETAB219_POS_DNPM_Plate-2_MDNAWGS14_rep1.featureXML


,charge,RT,mz,RTstart,RTend,quality,intensity
id,,,,,,,
9580361810607300402,1,61.796524,152.056168,54.532444,69.039505,2.660101,1.635787e+06
1198497665327712312,1,85.601051,153.040192,80.940842,90.342621,-4.545573,3.187437e+03
14561834521017206324,1,61.825218,155.080963,54.532444,69.039505,4.179129,2.525302e+05
1392807159208332361,1,61.232738,157.096588,53.253197,69.039505,0.950524,4.229514e+04
8719927500069282417,1,61.825600,158.091812,54.532444,69.039505,4.538174,3.799654e+05
...,...,...,...,...,...,...,...
4814547523342799275,3,56.009838,1482.843018,48.548748,58.175213,3.975889,2.401849e+05
3159795235816471004,3,59.875805,1483.177979,48.548748,70.226730,1.009578,1.584640e+06
14354688765166597198,2,63.236725,1485.509888,55.768665,72.628120,1.519182,2.749468e+05


FFMI_20210827_UMETAB219_POS_DNPM_Plate-2_MDNAWGS14_rep2.featureXML


...

,charge,RT,mz,RTstart,RTend,quality,intensity
id,,,,,,,
13827957299508051744,1,62.831272,152.056168,55.515396,70.156944,-1.779730,3.016066e+04
10177176204019348086,1,62.831360,155.080963,55.515396,70.156944,4.078637,1.934901e+05
2665139354868761074,1,62.831257,157.096588,55.515396,70.156944,4.051111,1.938711e+04
15553797221256980599,1,62.831272,158.091812,55.515396,70.156944,4.762939,1.776244e+06
15834760176572947576,1,292.991211,159.091064,285.836761,300.156616,-3.557896,6.691771e+03
...,...,...,...,...,...,...,...
11540701551856539020,3,52.647251,1482.843018,45.542580,55.515396,3.931542,4.255687e+04
7314053945385745425,3,60.831097,1483.177979,56.725086,68.949799,3.998992,8.815782e+04
6304223559691399761,2,63.283485,1485.509888,56.725086,71.362312,4.490026,4.659738e+05


FFMI_20210827_UMETAB219_POS_ISP2_Plate-2_MDNAWGS14_rep2.featureXML


,charge,RT,mz,RTstart,RTend,quality,intensity
id,,,,,,,
18305829263457869265,1,60.337032,152.056168,53.099388,67.737648,-1.755976,9.872725e+04
15851992003601902530,1,76.023224,153.040192,72.554497,83.347626,-3.420941,2.993516e+03
17074519437303712218,1,60.333485,155.080963,53.099388,67.737648,4.647839,1.883950e+05
14725209559210276962,1,60.336597,157.096588,53.099388,67.737648,4.007604,1.343551e+04
8440938669220168811,1,78.586952,158.091812,72.554497,84.534706,-1.250715,2.839973e+03
...,...,...,...,...,...,...,...
4403878866212871728,3,60.872658,1483.177979,56.783195,70.139503,4.030755,7.846965e+04
14440430011684610011,2,60.689110,1485.509888,53.099388,70.139503,4.696503,4.707206e+05
6982369948880004794,1,52.713772,1485.513306,44.017231,56.783195,3.881277,6.006521e+05


FFMI_20210827_UMETAB219_POS_ISP2_Plate-2_MDNAWGS14_rep3.featureXML


,charge,RT,mz,RTstart,RTend,quality,intensity
id,,,,,,,
17806343080640522621,1,61.236183,152.056168,54.287907,68.918221,-4.592659,9.799349e+04
1330555140745389982,1,95.550858,153.040192,88.074379,103.262917,-5.887690,5.471522e+03
1411516728387456180,1,61.294632,155.080963,53.027191,68.918221,0.699221,3.201624e+05
11970756054761642495,1,60.908886,157.096588,53.027191,68.918221,3.988412,2.442014e+04
16619539951047472698,1,62.848072,158.091812,55.533226,70.113480,4.760051,1.878474e+06
...,...,...,...,...,...,...,...
4262540228526693907,3,51.215118,1482.843018,44.377449,54.287907,1.518872,2.416276e+04
4032292776226716538,3,60.568966,1483.177979,53.027191,67.723534,4.583872,7.558447e+04
6643145562306594134,1,52.541622,1485.513306,44.377449,68.918221,4.069364,6.749305e+05


In [11]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

path= sorted(glob.glob('results/Requant/interim/*.featureXML'))

def plotDetectedFeatures3D(filename):
    fm = FeatureMap()
    fh = FeatureXMLFile()
    fh.load(filename, fm)

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    for feature in fm:
        color = next(ax._get_lines.prop_cycler)['color']
        # chromatogram data is stored in the subordinates of the feature
        for i, sub in enumerate(feature.getSubordinates()):
            retention_times = [x[0] for x in sub.getConvexHulls()[0].getHullPoints()]
            intensities = [int(y[1]) for y in sub.getConvexHulls()[0].getHullPoints()]
            mz = sub.getMetaValue('MZ')
            ax.plot(retention_times, intensities, zs = mz, zdir = 'x', color = color)
            if i == 0:
                ax.text(mz,retention_times[0], max(intensities)*1.02, feature.getMetaValue('label'), color = color)

    ax.set_ylabel('time (s)')
    ax.set_xlabel('m/z')
    ax.set_zlabel('intensity (cps)')
    plt.show()


In [ ]:
for filename in path:
    print(filename)
    plotDetectedFeatures3D(filename)

### Create a matrix for PCA

In [3]:
class ConsensusMapDF(ConsensusMap):
    def __init__(self):
        super().__init__()

    def get_intensity_df(self):
        labelfree = self.getExperimentType() == "label-free"
        filemeta = self.getColumnHeaders()  # type: dict[int, ColumnHeader]
        labels = list(set([header.label for header in
                           filemeta.values()]))  # TODO could be more efficient. Do we require same channels in all files?
        files = list(set([header.filename for header in filemeta.values()]))
        label_to_idx = {k: v for v, k in enumerate(labels)}
        file_to_idx = {k: v for v, k in enumerate(files)}

        def gen(cmap: ConsensusMap, fun):
            for f in cmap:
                yield from fun(f)

        if not labelfree:
            # TODO write two functions for LF and labelled. One has only one channel, the other has only one file per CF
            def extractRowBlocksChannelWideFileLong(f: ConsensusFeature):
                subfeatures = f.getFeatureList()  # type: list[FeatureHandle]
                filerows = defaultdict(lambda: [0] * len(labels))  # TODO use numpy array?
                for fh in subfeatures:
                    header = filemeta[fh.getMapIndex()]
                    row = filerows[header.filename]
                    row[label_to_idx[header.label]] = fh.getIntensity()
                return (f.getUniqueId(), filerows)

            def extractRowsChannelWideFileLong(f: ConsensusFeature):
                uniqueid, rowdict = extractRowBlocksChannelWideFileLong(f)
                for file, row in rowdict.items():
                    row.append(file)
                    yield tuple([uniqueid] + row)

            if len(labels) == 1:
                labels[0] = "intensity"
            dtypes = [('id', np.dtype('uint64'))] + list(zip(labels, ['f'] * len(labels)))
            dtypes.append(('file', 'U300'))
            # For TMT we know that every feature can only be from one file, since feature = PSM
            #cnt = 0
            #for f in self:
            #    cnt += f.size()

            intyarr = np.fromiter(iter=gen(self, extractRowsChannelWideFileLong), dtype=dtypes, count=self.size())
            return pd.DataFrame(intyarr).set_index('id')
        else:
            # Specialized for LabelFree which has to have only one channel
            def extractRowBlocksChannelLongFileWideLF(f: ConsensusFeature):
                subfeatures = f.getFeatureList()  # type: list[FeatureHandle]
                row = [0.] * len(files)  # TODO use numpy array?
                for fh in subfeatures:
                    header = filemeta[fh.getMapIndex()]
                    row[file_to_idx[header.filename]] = fh.getIntensity()
                yield tuple([f.getUniqueId()] + row)

            dtypes = [('id', np.dtype('uint64'))] + list(zip(files, ['f'] * len(files)))
            # cnt = self.size()*len(files) # TODO for this to work, we would need to fill with NAs for CFs that do not go over all files
            cnt = self.size()

            intyarr = np.fromiter(iter=gen(self, extractRowBlocksChannelLongFileWideLF), dtype=dtypes, count=cnt)
            return pd.DataFrame(intyarr).set_index('id')

    def get_metadata_df(self):
        def gen(cmap: ConsensusMap, fun):
            for f in cmap:
                yield from fun(f)

        def extractMetaData(f: ConsensusFeature):
            # subfeatures = f.getFeatureList()  # type: list[FeatureHandle]
            pep = f.getPeptideIdentifications()  # type: list[PeptideIdentification]
            if len(pep) != 0:
                hits = pep[0].getHits()
                if len(hits) != 0:
                    besthit = hits[0]  # type: PeptideHit
                    # TODO what else
                    yield f.getUniqueId(), besthit.getSequence().toString(), f.getCharge(), f.getRT(), f.getMZ(), f.getQuality()
                else:
                    yield f.getUniqueId(), None, f.getCharge(), f.getRT(), f.getMZ(), f.getQuality()
            else:
                yield f.getUniqueId(), None, f.getCharge(), f.getRT(), f.getMZ(), f.getQuality()

        cnt = self.size()

        mddtypes = [('id', np.dtype('uint64')), ('sequence', 'U200'), ('charge', 'i4'), ('RT', 'f'), ('mz', 'f'),
                    ('quality', 'f')]
        mdarr = np.fromiter(iter=gen(self, extractMetaData), dtype=mddtypes, count=cnt)
        return pd.DataFrame(mdarr).set_index('id')

In [4]:
feature_grouper = FeatureGroupingAlgorithmKD()

consensus_map = ConsensusMapDF()
file_descriptions = consensus_map.getColumnHeaders()

input_feature_files = sorted(glob.glob('results/Requant/interim/*.featureXML'))
print(input_feature_files)

feature_maps = []
for featurexml_file in input_feature_files:
    fmap = FeatureMap()
    FeatureXMLFile().load(featurexml_file, fmap)
    feature_maps.append(fmap)

for i, feature_map in enumerate(feature_maps):
    file_description = file_descriptions.get(i, ColumnHeader())
    file_description.filename = os.path.basename(feature_map.getMetaValue('spectra_data')[0].decode())[34:-5]
    file_description.size = feature_map.size()
    file_descriptions[i] = file_description

feature_grouper.group(feature_maps, consensus_map)
consensus_map.setColumnHeaders(file_descriptions)

Consensus_file= os.path.join("results", "", "Requant", "","interim", "", 'consensus' + ".consensusXML")
ConsensusXMLFile().store(Consensus_file, consensus_map)

# get intensities as a DataFrame
intensities = consensus_map.get_intensity_df()

# get meta data as DataFrame
meta_data = consensus_map.get_metadata_df()[['RT', 'mz', 'charge']]

# you can concatenate these two for a "result" DataFrame
result = pd.concat([meta_data, intensities], axis=1)

# if you don't need labeled index, remove it (and/or save with index = False)
result.reset_index(drop=True, inplace=True)

# store as tsv file
result.to_csv('results/Requant/FeatureMatrix.tsv', sep = '\t', index = False)

['results/Requant/interim/FFMI_20210827_UMETAB219_POS_DNPM_Plate-2_MDNAWGS14_blank.featureXML', 'results/Requant/interim/FFMI_20210827_UMETAB219_POS_DNPM_Plate-2_MDNAWGS14_rep1.featureXML', 'results/Requant/interim/FFMI_20210827_UMETAB219_POS_DNPM_Plate-2_MDNAWGS14_rep2.featureXML', 'results/Requant/interim/FFMI_20210827_UMETAB219_POS_DNPM_Plate-2_MDNAWGS14_rep3.featureXML', 'results/Requant/interim/FFMI_20210827_UMETAB219_POS_FPY12_Plate-2_MDNAWGS14_blank.featureXML', 'results/Requant/interim/FFMI_20210827_UMETAB219_POS_FPY12_Plate-2_MDNAWGS14_rep1.featureXML', 'results/Requant/interim/FFMI_20210827_UMETAB219_POS_FPY12_Plate-2_MDNAWGS14_rep2.featureXML', 'results/Requant/interim/FFMI_20210827_UMETAB219_POS_FPY12_Plate-2_MDNAWGS14_rep3.featureXML', 'results/Requant/interim/FFMI_20210827_UMETAB219_POS_ISP2_Plate-2_MDNAWGS14_blank.featureXML', 'results/Requant/interim/FFMI_20210827_UMETAB219_POS_ISP2_Plate-2_MDNAWGS14_rep1.featureXML', 'results/Requant/interim/FFMI_20210827_UMETAB219_POS

### Compare the preprocessed and requantified numbers of features (missing values):

In [3]:
from pyopenms import *
import pandas as pd
import os
import glob

Preprocessed= glob.glob("results/features/*.csv")
Requantified= glob.glob("results/Requant/*.csv")

for table in Preprocessed:
    df1= pd.read_csv(table)
    feature_no= len(df1)

    for matrix in Requantified:
        df2= pd.read_csv(matrix)
        requant_no= len(df2)

        if os.path.basename(table)== os.path.basename(matrix):
            print(os.path.basename(matrix))
            print("Preprocessed:", feature_no, "Requantified:", requant_no)
   

features_20210827_UMETAB219_POS_ISP2_Plate-2_MDNAWGS14_blank.csv
Preprocessed: 1267 Requantified: 5294
features_20210827_UMETAB219_POS_DNPM_Plate-2_MDNAWGS14_rep1.csv
Preprocessed: 623 Requantified: 4467
features_20210827_UMETAB219_POS_DNPM_Plate-2_MDNAWGS14_rep3.csv
Preprocessed: 556 Requantified: 4451
features_20210827_UMETAB219_POS_DNPM_Plate-2_MDNAWGS14_rep2.csv
Preprocessed: 543 Requantified: 4923
features_20210827_UMETAB219_POS_ISP2_Plate-2_MDNAWGS14_rep2.csv
Preprocessed: 523 Requantified: 4068
features_20210827_UMETAB219_POS_FPY12_Plate-2_MDNAWGS14_blank.csv
Preprocessed: 3008 Requantified: 5045
features_20210827_UMETAB219_POS_ISP2_Plate-2_MDNAWGS14_rep3.csv
Preprocessed: 532 Requantified: 3807
features_20210827_UMETAB219_POS_ISP2_Plate-2_MDNAWGS14_rep1.csv
Preprocessed: 636 Requantified: 3936
features_20210827_UMETAB219_POS_FPY12_Plate-2_MDNAWGS14_rep3.csv
Preprocessed: 2436 Requantified: 6625
features_20210827_UMETAB219_POS_FPY12_Plate-2_MDNAWGS14_rep2.csv
Preprocessed: 2350 